In [1]:
import numpy as np
import scipy.optimize as optim
import pandas as pd
import math
from sklearn.metrics import accuracy_score

In [2]:
# icml

In [3]:
data=pd.read_csv('/Users/apple/Downloads/compas-scores-two-years.csv')
data=data[(data['race']=='African-American') | (data['race']=='Caucasian')]
data['race'].loc[data['race']=='African-American']= 1
data['race'].loc[data['race']=='Caucasian']= 0

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
data.columns

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

In [5]:
data=pd.DataFrame(data,columns=['race','juv_fel_count', 'decile_score','juv_misd_count','two_year_recid'])
data=data.iloc[:20]
from sklearn.model_selection import train_test_split
data_train,data_val=train_test_split(data,random_state=0)

In [6]:
data.head()

,race,juv_fel_count,decile_score,juv_misd_count,two_year_recid
1,1,0,3,0,1
2,1,0,4,0,1
3,1,0,8,1,0
6,0,0,6,0,1
8,0,0,1,0,0


In [7]:
data.iloc[0][0]

1

In [8]:
# distance - d(x_n, v_k, alpha)
def distance(X, v, alpha):
    # X is the entire training set (? or any dataset)
    # *****should be a dataframe*****
    # - has N samples (N rows)
    # - each samples has a length of D (number of features?)
    # v is a list of vectors, where each vector v_k has a length of D
    # alpha is a list of weights, each alpha_i is the weight for some feature
    N = X.shape[0]
    D = X.shape[1]
    
    # note how to store v
    K = len(v)

    # initialize a list of distance 
    # this should have a length N
    res = np.zeros((N, K))
    
    for n in range(N):
        for k in range(K):
            for d in range(D):         
                res[n, k] += alpha[d]*(X.iloc[n][d] - v[k, d])**2
    
    return res

In [9]:
# this function calculates M_nk
# M_nk = P(Z=k|x_n), which is the probablity that x_n maps to v_k
# x_n is a vector of length D
# v is list of vectors, where each vector v_k has a length of D
def M_nk(dist, k):
    N = dist.shape[0]
    K = dist.shape[1]
    M_nk = np.zeros((N, K))
    expo_res = np.zeros((N, K))
    
    for n in range(N):
        deno = 0
        for k in range(K):
            expo_res[n, k] = math.exp((-1)*dist[n, k])
            deno += expo_res[n, k]
        for k in range(K):
            M_nk[n, k] = expo_res[n, k] / deno
            # raise exception
    return M_nk

In [10]:
# this function calculates M_k
# X is the entire training set (? or any dataset)
# *****should be a dataframe*****
# - has N samples (N rows)
# - each samples has a length of D (number of features?)
# v is a list of vectors, where each vector v_k has a length of D
# alpha is a list of weights, each alpha_i is the weight for some feature, length of D
def M_k(X, M_nk, k):
    N = X.shape[0]
    K = M_nk.shape[1]
    M_k = np.zeros(K)
    
    for k in range(K):
        for n in range(N):
            M_k[k] += M_nk[n, k]
        M_k[k] = M_k[k]/N
        
    return M_k


In [11]:
# this function calculates the reconstruction of x_n
# ???????????? check alpha_i #######
def x_n_hat(X, M_nk, v):
    N = M_nk.shape[0]
    D = X.shape[1]
    K = M_nk.shape[1]
    x_n_hat = np.zeros((N, D))
    L_x = 0
    
    for n in range(N):
        for d in range(D):
            for k in range(K):
                # note the format of v
                x_n_hat[n, d] += M_nk[n, k]*v[k, d]
        L_x += (X.iloc[n][d] - x_n_hat[n, d])**2
        
    # Check the length of the L_x and x_n_hat
    
    return x_n_hat, L_x

In [12]:
# this function calculates the prediction for y_n
# w is a list of weights of length K
def y_n_hat(M_nk, w, y):
    N = M_nk.shape[0]
    K = M_nk.shape[1]
    y_n_hat = np.zeros(N)
    L_y = 0
    
    for n in range(N):
        for k in range(K):
            y_n_hat[n] += M_nk[n, k]*w[k]
        
        # what if y_n_hat < 0
        
        # print((-1)*y[n]*np.log(y_n_hat[n]))
        L_y += (-1)*y.iloc[n]*np.log(y_n_hat[n]) - (1 - y.iloc[n])*np.log(1 - y_n_hat[n])
        
    return y_n_hat, L_y

In [13]:
def L(param, sen_df, nsen_df, sen_y, nonsen_y, K, A_z, A_x, A_y):
    
    sen_N, sen_D = sen_df.shape
    nsen_N, nsen_D = nsen_df.shape
    
#     alpha_sen_1=np.random.random_sample((sen_D,))
#     alpha_nsen_1=np.random.random_sample((sen_D,))
#     alpha_sen=alpha_sen_1/sum(alpha_sen_1)
#     alpha_nsen=alpha_nsen_1/sum(alpha_nsen_1)
#     w_1=np.random.random_sample((K,))
#     w=w_1/sum(w_1)
    
#     v=np.random.random((K, sen_D))

    alpha_sen = param[:sen_D]
    alpha_nsen = param[sen_D : 2 * sen_D]
    w = param[2 * sen_D : (2 * sen_D) + K]
    v = np.matrix(param[(2 * sen_D) + K:]).reshape((K, sen_D))
    
    
    para_values = []
    f_value = []

    # for i in range(1, 2):
    dist_sen = distance(sen_df, v, alpha_sen)
    dist_nsen = distance(nsen_df, v, alpha_nsen)        

    M_nk_sen = M_nk(dist_sen, K)
    M_nk_nsen = M_nk(dist_nsen, K)

    M_k_sen = M_k(sen_df, M_nk_sen, K)
    M_k_nsen = M_k(nsen_df, M_nk_nsen, K)

    L_z = 0
    for k in range(K):
        L_z += abs(M_k_sen[k] - M_k_nsen[k])

    x_n_hat_sen, L_x_sen = x_n_hat(sen_df, M_nk_sen, v)
    x_n_hat_nsen, L_x_nsen = x_n_hat(nsen_df, M_nk_nsen, v)

    L_x = L_x_sen + L_x_nsen

       # print(sen_y.iloc[0])
    y_hat_sen, L_y_sen = y_n_hat(M_nk_sen, w, sen_y)
        
    y_hat_nsen, L_y_nsen = y_n_hat(M_nk_nsen, w, nonsen_y)

    L_y = L_y_sen + L_y_nsen

    metric = A_z*L_z + A_x*L_x + A_y*L_y

    return metric


In [36]:
def predic_threshold(preds):
    for i in range(len(preds)):
        if preds[i] >= 0.5:
            preds[i] = 1
        else:
            preds[i] = 0
    return preds

In [19]:
def cal_pred(params, D, K, sen_dt, nsen_dt, sen_label, nsen_label):
    
    best_alpha_sen = params[:D]
    best_alpha_nsen = params[D : 2 * D]
    best_w = params[2 * D : (2 * D) + K]
    best_v = np.matrix(params[(2 * D) + K:]).reshape((K, D))
    
    best_dist_sen = distance(sen_dt, best_v, best_alpha_sen)
    best_dist_nsen = distance(nsen_dt, best_v, best_alpha_nsen) 
    
    best_M_nk_sen = M_nk(best_dist_sen, K)
    best_M_nk_nsen = M_nk(best_dist_nsen, K)
    
    y_hat_sen, L_y_sen = y_n_hat(best_M_nk_sen, best_w, sen_label)
    y_hat_nsen, L_y_nsen = y_n_hat(best_M_nk_nsen, best_w, nsen_label)
    
    return y_hat_sen, y_hat_nsen

In [47]:
def cal_calibr(y_pred_sen, y_pred_nsen, y_sen_label, y_nsen_label):
    converted_y_hat_sen = predic_threshold(y_pred_sen)
    converted_y_hat_nsen = predic_threshold(y_pred_nsen)

    y_pred_sen = pd.DataFrame(converted_y_hat_sen)
    y_pred_nsen = pd.DataFrame(converted_y_hat_nsen)

#     print(y_pred_sen)
#     print(y_pred_nsen)
#     print(y_sen_label)
#     print(y_nsen_label)
    
    acc_sen = accuracy_score(y_sen_label, y_pred_sen)
    acc_nsen = accuracy_score(y_nsen_label, y_pred_nsen)
    
    all_labels = y_sen_label.append(y_nsen_label)
    all_preds = y_pred_sen.append(y_pred_nsen)
    
    total_accuracy = accuracy_score(all_preds, all_labels)
    
    print("The accuracy for the entire dataset is: ", total_accuracy)
    print("The accuracy for xxx group is: ", acc_sen)
    print("The accuracy for xxx group is: ", acc_nsen)
    print("The calibration is: ", acc_sen-acc_nsen)

In [41]:
###### ?????????????
def LFR(training_data, val_data, y_name, sen_variable_name, K, A_z, A_x, A_y):
    # divide the training set into sensitive & nonsensitive group
    sen_training = training_data[training_data[sen_variable_name]==0]
    nsen_training = training_data[training_data[sen_variable_name]==1]
    
    # divide the validation set into sensitive & nonsensitive group
    sen_val = val_data[val_data[sen_variable_name]==0]
    nsen_val = val_data[val_data[sen_variable_name]==1]
    
    # remove sensitive variable in the sensitive training and validation group
    sen_training=sen_training.drop(columns=[sen_variable_name])
    sen_val=sen_val.drop(columns=[sen_variable_name])

    # remove sensitive variable in the nonsensitive training and validation group    
    nsen_training = nsen_training.drop(columns=[sen_variable_name])
    nsen_val = nsen_val.drop(columns=[sen_variable_name])
    
    # assign y labels for sensitive training group
    y_sen_training = sen_training[y_name]
    sen_training = sen_training.drop(columns=[y_name])
    
    # assign y labels for sensitive validation group
    y_sen_val = sen_val[y_name]
    sen_val = sen_val.drop(columns=[y_name])
    
    # assign y labels for nonsensitive training group
    y_nsen_training = nsen_training[y_name]
    nsen_training = nsen_training.drop(columns=[y_name])
    
    # assign y labels for nonsensitive validation group
    y_nsen_val = nsen_val[y_name]
    nsen_val = nsen_val.drop(columns=[y_name])
    
    alpha_sen_1=np.random.random_sample((sen_training.shape[1],))
    alpha_nsen_1=np.random.random_sample((nsen_training.shape[1],))
    alpha_sen=alpha_sen_1/sum(alpha_sen_1)
    alpha_nsen=alpha_nsen_1/sum(alpha_nsen_1)
    w_1=np.random.random_sample((K,))
    w=w_1/sum(w_1)
    v=np.random.random((K, sen_training.shape[1]))
    
    initial = []
    initial.extend(alpha_sen)
    initial.extend(alpha_nsen)
    initial.extend(w)
    
    for item in v:
        initial.extend(item)
    initial = np.array(initial)

    bound=[]

    for d in range(sen_training.shape[1]):
        bound.append((0, 1))

    for d in range(nsen_training.shape[1]):
        bound.append((0, 1))

    for k in range(K):
        bound.append((0, 1))

    for k in range(K):
        for d in range(sen_training.shape[1]):
            bound.append((None, None))
    
    
    para, min_L, d = optim.fmin_l_bfgs_b(L, x0=initial, epsilon=1e-5, 
                                         args=(sen_training, nsen_training, y_sen_training, 
                                               y_nsen_training, K, A_z, A_x, A_y), 
                                         bounds = bound, approx_grad=True, 
                                         maxfun=150000, maxiter=150000)
    
    y_hat_sen_tr, y_hat_nsen_tr = cal_pred(para, sen_training.shape[1], K, sen_training, 
             nsen_training, y_sen_training, y_nsen_training)

    print("For the training set:")
    cal_calibr(y_hat_sen_tr, y_hat_nsen_tr, y_sen_training, y_nsen_training)
    
    print("++++++++++++++++++++++++++++")
    print("For the validation set:")
    
    y_hat_sen_val, y_hat_nsen_val = cal_pred(para, sen_val.shape[1], K, sen_val, 
             nsen_val, y_sen_val, y_nsen_val)  
    print("For the validation set:")
    cal_calibr(y_hat_sen_val, y_hat_nsen_val, y_sen_val, y_nsen_val)    
    
    return  para

In [48]:
LFR(data_train, data_val, 'two_year_recid', 'race', 2, 0.2, 0.4, 0.4)

For the training set:
The accuracy for the entire dataset is:  0.7333333333333333
The accuracy for xxx group is:  0.7142857142857143
The accuracy for xxx group is:  0.75
The calibration is:  -0.0357142857142857
++++++++++++++++++++++++++++
For the validation set:
For the validation set:
The accuracy for the entire dataset is:  0.4
The accuracy for xxx group is:  0.25
The accuracy for xxx group is:  1.0
The calibration is:  -0.75


array([ 0.14706479,  0.        ,  0.67007669,  0.7980738 ,  0.15606314,
        1.        ,  0.        ,  1.        ,  0.87961552,  0.47579889,
        1.14292234,  0.21571518,  0.08433486, -0.46668309])